In [2]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('2020_10_seoul-bus.csv',encoding='cp949')

df_stopcode = df['버스정류장ARS번호'] # 시각화 때 좌표값 조회를 위한 정류장 코드 저장

df.drop(df.columns[[0,2,3,4,54]], axis = 1, inplace = True)

df["bus_Term"] = "NA"
df["per_Hour"] = "NA"


df2 = pd.read_csv('ct.csv',encoding='cp949')


a=0
while (a<128):
    df.loc[df["노선번호"] == df2.iloc[a,0], "bus_Term"] = df2.iloc[a,1]
    a+=1
    
b=0
while (b<128):
    df.loc[df["노선번호"] == df2.iloc[b,0], "per_Hour"] = df2.iloc[b,2]
    b+=1
for i in range(2,50,2):##정류장에서의 시간대별 승차인원 수 - 정류장에서의 시간대별 하차인원수=포화도의 변화량
    df.iloc[0:,i] = df.iloc[0:,i]-df.iloc[0:,i+1]
    
for j in range(3,27):##승차인원수 열들에 포화도의 변화량을 저장했으므로 하차인원열 드랍
    df = df.drop(df.columns[j], axis='columns')
    

df3 = df.drop(df.columns[[0,1,26,27]], axis='columns')##따로 숫자 계산을 위해 문자데이터로 된 열 드랍하고 데이터프레임 생성
df3['버스한대 당 정류장의 하루평균 포화도의 증가량'] = df3.sum(axis=1)##정류장 마다 시간대 별 포화도의 증가량을 다합침(버스한대 당 정류장의 하루평균 포화도의 증가량이 아직은 아님 )
result_data = pd.concat([df["per_Hour"],df3["버스한대 당 정류장의 하루평균 포화도의 증가량"]], axis=1)##결과 데이터라는 데이터프레임 생성훈 필요한 데이터 열 추가
result_data = result_data.div(df["per_Hour"],axis=0)##버스한대 당 정류장의 하루평균 포화도의 증가량 데이터를 시간당 버스 대수로 나눔
result_data = result_data.div(24,axis=0)##시간별로 다더했으므로 24로 나눔  즉, 버스대수와 24로 나누었으므로  정류장에서의 버스 한대 당 하루평균 포화도의 증가량이됌
del result_data["per_Hour"]##필요없는 열 삭제
result_data = pd.concat([df["노선번호"],result_data], axis=1)#노선 번호 열 추가

df_vis = result_data #시각화를  위한 데이터프레임 준비
df_vis = pd.concat([df_stopcode,result_data], axis=1) #정류장 번호 추가(정류장 위치값을 읽어올 때 사용)(시각화)



for i in range(0,12680):##정류장 별 평균 포화도의 증가량 열 중에 음수인 값들은 드랍
    if(result_data.loc[i,"버스한대 당 정류장의 하루평균 포화도의 증가량"]<0):
        result_data = result_data.drop(i)

result_data = result_data.reset_index()##인덱스 다시 1,2,3...으로 초기화
del result_data["index"]##필요없는 열 삭제

busnum = 0 ## 버스 번호 저장할 변수
dict_data = {}## key : 버스 번호 와 value : 버스 정류장별 평균 포화도의 증가량의 양수값들을  다 더해서 평균낸 값
total = 0 ## 더한 값 저장 변수
j = 0## 버스마다 평균 포화도의 양수인 값을 갯수 저장 변수

for i in range(0,6224):##버스 별로 평균 포화도의 증가량의 양수값들을  다 더해서 평균낸 값을 구한후 딕셔너리에 key value 형태로 저장
    if(busnum == 0):
        busnum = result_data.loc[i,"노선번호"]
    elif(busnum != result_data.loc[i,"노선번호"]):
        dict_data[busnum] = total / j
        busnum = result_data.loc[i,"노선번호"]
        total = 0
        j = 0
        
    if(result_data.loc[i,"노선번호"]==busnum):
        total += result_data.loc[i,"버스한대 당 정류장의 하루평균 포화도의 증가량"]
        j += 1
        if(i == 6223):
            dict_data[busnum] = total / j
            
result_data=pd.Series(dict_data)## 딕셔너리를 시리즈 데이터로 생성

In [3]:
fo = pd.read_csv('bus_loc.csv',encoding='cp949') #버스 정류장 좌표값 담은 데이터
top_bus = result_data.sort_values(ascending = False).head(5) #상위 5개 버스
under_bus = result_data.sort_values(axis=0).head(5) #하위 5개 버스


a = top_bus.index # 상위 5개 버스 index값
aa = under_bus.index
is_topbus = df_vis['노선번호'].isin(a)
is_underbus = df_vis['노선번호'].isin(aa)

b = df_vis[is_topbus].copy() #상위 5개 버스 정보만 가진 데이터프레임
bb = df_vis[is_underbus].copy()#하위 5개 버스 정보만 가진 데이터프레임

b.rename(columns= {'버스정류장ARS번호' : '정류소번호'}, inplace = True)
bb.rename(columns= {'버스정류장ARS번호' : '정류소번호'}, inplace = True)
busstop = pd.merge(b, fo) #좌표값 담은 데이터와 상위 포화도 값 가진 데이터 합치기
busstop2 = pd.merge(bb, fo) #좌표값 담은 데이터와 하위 포화도 값 가진 데이터 합치기

In [5]:
#포화도 상위 5개 버스 시각화
import folium
from folium import plugins
from folium.plugins import MarkerCluster

map_hs = folium.Map((37.579183, 126.996566), zoom_start = 12) 

mc = MarkerCluster()
busstop.rename(columns= {'버스한대 당 정류장의 하루평균 포화도의 증가량' : '포화도증가량'}, inplace = True)
colors = ["#000", '#f00', '#080','#0c0', '#f80' ]

for i in range(len(a)):   #버스 번호별로 분리
    is_busstop = busstop['노선번호'].str.contains(a[i])
    stop_temp = busstop[is_busstop]

    for row in stop_temp.itertuples():#각 정류소 좌표, 노선번호, 정류소 이름넘겨 마커로 표시
        mc.add_child(folium.Marker(location = [row.Y좌표, row.X좌표], tooltip = str(row.노선번호)+ '번\n' + row.정류소명, popup = row.포화도증가량))
        map_hs.add_child(mc)
    coords = stop_temp[["Y좌표","X좌표"]]
    plugins.PolyLineOffset(coords, color=colors[i], opacity=1, offset=-5).add_to(map_hs) #마커끼리 연결
    
map_hs.save('./bus_top5.html')


In [7]:
#포화도 하위 5개 버스 시각화
map_hs3 = folium.Map((37.579183, 126.996566), zoom_start = 12) 

mc3 = MarkerCluster()
busstop2.rename(columns= {'버스한대 당 정류장의 하루평균 포화도의 증가량' : '포화도증가량'}, inplace = True)
colors = ["#000", '#f00', '#080','#0c0', '#f80' ]


for i in range(len(aa)):    #버스 번호별로 분리
    is_busstop = busstop2['노선번호'].str.contains(aa[i])
    stop_temp = busstop2[is_busstop]


    for row in stop_temp.itertuples():#각 정류소 좌표, 노선번호, 정류소 이름넘겨 마커로 표시
        mc3.add_child(folium.Marker(location = [row.Y좌표, row.X좌표], tooltip = str(row.노선번호)+ '번\n' + row.정류소명, popup = row.포화도증가량))
        map_hs3.add_child(mc3)
    coords = stop_temp[["Y좌표","X좌표"]]
    plugins.PolyLineOffset(coords, color=colors[i], opacity=1, offset=-5).add_to(map_hs3)  #마커끼리 연결
    
map_hs3.save('./bus_under5.html')


In [12]:
#상위 5개 버스 대상 포화도 60이상 값 가지는 정류장 출력
map_hs2 = folium.Map((37.579183, 126.996566), zoom_start = 12)

mc2 = MarkerCluster()

busstop.rename(columns= {'버스한대 당 정류장의 하루평균 포화도의 증가량' : '포화도증가량'}, inplace = True)
over_num = busstop['포화도증가량'] >= 60 # 조회하고 싶은 값 입력
over_topbus = busstop[over_num]

for row in over_topbus.itertuples():#각 정류소 좌표, 노선번호, 정류소 이름넘겨 마커로 표시
    mc2.add_child(folium.Marker(location = [row.Y좌표, row.X좌표], tooltip = str(row.노선번호)+ '번\n' + row.정류소명, popup = row.포화도증가량))
    map_hs2.add_child(mc2)
coords = over_topbus[["Y좌표","X좌표"]]
plugins.PolyLineOffset(coords, color='#000', opacity=1, offset=-5).add_to(map_hs2)  #마커끼리 연결
    
map_hs2.save('./bus_stops_over60_top5.html')

In [14]:
#상위 5개 버스 대상 포화도 -60 이하 값 가지는 정류장 출력
map_hs2 = folium.Map((37.579183, 126.996566), zoom_start = 12)

mc2 = MarkerCluster()

busstop.rename(columns= {'버스한대 당 정류장의 하루평균 포화도의 증가량' : '포화도증가량'}, inplace = True)
over_num = busstop['포화도증가량'] <= -60 # 조회하고 싶은 값 입력
over_topbus = busstop[over_num]

for row in over_topbus.itertuples():#각 정류소 좌표, 노선번호, 정류소 이름넘겨 마커로 표시
    mc2.add_child(folium.Marker(location = [row.Y좌표, row.X좌표], tooltip = str(row.노선번호)+ '번\n' + row.정류소명, popup = row.포화도증가량))
    map_hs2.add_child(mc2)
coords = over_topbus[["Y좌표","X좌표"]]
plugins.PolyLineOffset(coords, color='#000', opacity=1, offset=-5).add_to(map_hs2)  #마커끼리 연결
    
map_hs2.save('./bus_stops_under-60_top5.html')

In [18]:
#상위 5개 버스 대상 포화도 이상 조회(60 조회시 60 이상 표시)

num1 = input("원하시는 포화도의 값을 입력하세요. : ")

num1 = int(num1)
map_hs2 = folium.Map((37.579183, 126.996566), zoom_start = 12)

mc2 = MarkerCluster()

busstop.rename(columns= {'버스한대 당 정류장의 하루평균 포화도의 증가량' : '포화도증가량'}, inplace = True)
over_num = busstop['포화도증가량'] >= num1 # 조회하고 싶은 값 입력
over_topbus = busstop[over_num]

for row in over_topbus.itertuples():#각 정류소 좌표, 노선번호, 정류소 이름넘겨 마커로 표시
    mc2.add_child(folium.Marker(location = [row.Y좌표, row.X좌표], tooltip = str(row.노선번호)+ '번\n' + row.정류소명, popup = row.포화도증가량))
    map_hs2.add_child(mc2)
coords = over_topbus[["Y좌표","X좌표"]]
plugins.PolyLineOffset(coords, color='#000', opacity=1, offset=-5).add_to(map_hs2)  #마커끼리 연결
    
map_hs2.save('./bus_stops_over_top5.html')# 조회후 시각화 데이터 저장(조회 결과는 파일로 확인하시면 됩니다.)

원하시는 포화도의 값을 입력하세요. : 80


In [21]:
#상위 5개 버스 대상 포화도 이하 조회(-60 조회시 -60 이하 표시)
map_hs2 = folium.Map((37.579183, 126.996566), zoom_start = 12)
num2 = input("원하시는 포화도의 값을 입력하세요. : ")
num2 = int(num2)
mc2 = MarkerCluster()

busstop.rename(columns= {'버스한대 당 정류장의 하루평균 포화도의 증가량' : '포화도증가량'}, inplace = True)
over_num = busstop['포화도증가량'] <= num2 # 조회하고 싶은 값 입력
over_topbus = busstop[over_num]

for row in over_topbus.itertuples():#각 정류소 좌표, 노선번호, 정류소 이름넘겨 마커로 표시
    mc2.add_child(folium.Marker(location = [row.Y좌표, row.X좌표], tooltip = str(row.노선번호)+ '번\n' + row.정류소명, popup = row.포화도증가량))
    map_hs2.add_child(mc2)
coords = over_topbus[["Y좌표","X좌표"]]
plugins.PolyLineOffset(coords, color='#000', opacity=1, offset=-5).add_to(map_hs2)  #마커끼리 연결
    
map_hs2.save('./bus_stops_under_top5.html')# 조회후 시각화 데이터 저장(조회 결과는 파일로 확인하시면 됩니다.)

원하시는 포화도의 값을 입력하세요. : -80
